<a href="https://colab.research.google.com/github/biglool/TFG_ALZEIMER/blob/main/Experiment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
TFG_DATA= '/content/drive/MyDrive/TFG_DATA/'
DATASET = 'ADNI_PROC/'
DS_INFO ='Info.csv'
PROBLEMTYPE = 'CNvsMCI'

Mounted at /content/drive


In [ ]:
!rm -rf tfg
!git clone -l -s https://github.com/biglool/TFG_ALZEIMER/ tfg


Cloning into 'tfg'...
remote: Enumerating objects: 465, done.
remote: Counting objects: 100% (465/465), done.
remote: Compressing objects: 100% (464/464), done.
remote: Total 465 (delta 214), reused 15 (delta 0), pack-reused 0
Receiving objects: 100% (465/465), 96.55 KiB | 2.76 MiB/s, done.
Resolving deltas: 100% (214/214), done.


In [ ]:
import torch
from tfg.code import database,training, trainingSteps
from tabulate import tabulate
import os


In [ ]:
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
#torch.cuda.memory_summary(device=None)

1
Tesla P100-PCIE-16GB


# **Part 1**: Busquem les configuracions.

In [ ]:
configs=[]
i=0
for net in ['Densenet121','Densenet201','Resnet18','Resnet50','Resnet152']:
  for batch in [16,32,64]:
    j=0
    for lr in [0.0001,0.00005,0.00001]:
      i=i+1
      j+=1
      nom=net+"_"+str(batch)+"_"+str(j)+".pt"
      configs.append([i,nom,net,batch,lr])


In [ ]:

EXPERIMENTFOLDER ='hiper2/'
num_epochs = 140
paciencia=12
carpeta = TFG_DATA+PROBLEMTYPE+"/"+EXPERIMENTFOLDER

dataset = database.ADNI_Dataset( DS_INFO ,TFG_DATA + DATASET)
cut=dataset.cut_id
dataset.setProblem(PROBLEMTYPE)
dataset.setCut(cut)

resultats=[]
train_idx,valid_idx,test_idx = trainingSteps.getSplits(dataset)

for conf in configs:

  nom=conf[1]
  model= conf[2]
  batch_size=conf[3]
  lr =  conf[4]

  print("Training...:"+carpeta+ nom )
  print("Configuracio:")
  print(conf)
  if not os.path.exists(carpeta+nom):
    training.createExperiment(carpeta,nom,PROBLEMTYPE,cut,train_idx,test_idx,model,lr,batch_size,valid_idx,paciencia,num_epochs)
    training.trainExperiment(carpeta, nom, device, dataset, earlyStop=True)

  print("Resultat:") 
  resultat=training.validateModel(carpeta, nom, device, dataset, verbose=True)
  resultats.append(conf+resultat)

print("Resultats finals:")
print(tabulate(resultats, headers=["N","nom","Model", "Batch","lr","Epoch","Loss", "Acc","BAcc","SEN","SPE","PRE","F1","AUC"]))


Training...:/content/drive/MyDrive/TFG_DATA/CNvsMCI/hiper2/Densenet121_16_1.pt
Configuracio:
[1, 'Densenet121_16_1.pt', 'Densenet121', 16, 0.0001]
Resultat:
epoch:
6
Loss: 0.438 | Acc: 0.778 | BAcc: 0.776 | SEN: 0.809 | SPE: 0.744 | PRE: 0.776 | F1: 0.792 | AUC: 0.776
Training...:/content/drive/MyDrive/TFG_DATA/CNvsMCI/hiper2/Densenet121_16_2.pt
Configuracio:
[2, 'Densenet121_16_2.pt', 'Densenet121', 16, 5e-05]
Resultat:
epoch:
13
Loss: 0.423 | Acc: 0.833 | BAcc: 0.833 | SEN: 0.851 | SPE: 0.814 | PRE: 0.833 | F1: 0.842 | AUC: 0.833
Training...:/content/drive/MyDrive/TFG_DATA/CNvsMCI/hiper2/Densenet121_16_3.pt
Configuracio:
[3, 'Densenet121_16_3.pt', 'Densenet121', 16, 1e-05]
Resultat:
epoch:
24
Loss: 0.445 | Acc: 0.767 | BAcc: 0.767 | SEN: 0.766 | SPE: 0.767 | PRE: 0.783 | F1: 0.774 | AUC: 0.767
Training...:/content/drive/MyDrive/TFG_DATA/CNvsMCI/hiper2/Densenet121_32_1.pt
Configuracio:
[4, 'Densenet121_32_1.pt', 'Densenet121', 32, 0.0001]
Resultat:
epoch:
7
Loss: 0.440 | Acc: 0.767 | 

# **Part 2:**Cross-validem les configuracions seleccionades.

In [ ]:

configs=[['Densenet121',32,0.0001,26],
         ['Densenet201',32,0.0001,6],
         ['Resnet18',64,0.00005,11],
         ['Densenet201',64,0.00005,11],
         ['Resnet50',16,0.0001,18],
         ['Resnet18',32,0.00005,8],
         ['Densenet121',32,0.00001,38],
         ['Resnet18',16,0.0001,27],
         ['Densenet201',32,0.0001,21],
         ['Densenet201',16,0.0001,5]]

      
EXPERIMENTFOLDER ='5fold/'
carpeta = TFG_DATA+PROBLEMTYPE+"/"+EXPERIMENTFOLDER

dataset = database.ADNI_Dataset( DS_INFO ,TFG_DATA + DATASET)
cut=dataset.cut_id
dataset.setProblem(PROBLEMTYPE)
dataset.setCut(cut)

folds=trainingSteps.getKfoldSlits(dataset)

for conf in configs:

  print("Training...:"+carpeta )
  print("Configuracio:")
  print(conf)  

  fold=0

  model= conf[0]
  batch_size=conf[1]
  lr =  conf[2]
  max_epoch = conf[3]
  ex_carpeta = carpeta + model+"_"+str(batch_size)+"_"+str(max_epoch) + "/"

  for train_idx,test_idx in folds:

    fold=fold+1
    nom = "fold"+ str(fold) +".pt"
    if not os.path.exists(ex_carpeta+nom):
      training.createExperiment(ex_carpeta,nom,PROBLEMTYPE,cut,train_idx,test_idx,model,lr,batch_size,max_epoch=max_epoch)
      training.trainExperiment(ex_carpeta, nom, device, dataset, verbose=False)

  print("Resultat:") 
  training.crossValidate(ex_carpeta, device, dataset, verbose=True)


Training...:/content/drive/MyDrive/TFG_DATA/CNvsMCI/5fold/
Configuracio:
['Densenet121', 32, 0.0001, 26]
Resultat:
Fold:1
epoch:
26
Loss: 6.698 | Acc: 0.475 | BAcc: 0.508 | SEN: 0.016 | SPE: 1.000 | PRE: 1.000 | F1: 0.031 | AUC: 0.508
Fold:2
epoch:
26
Loss: 0.959 | Acc: 0.717 | BAcc: 0.714 | SEN: 0.750 | SPE: 0.679 | PRE: 0.727 | F1: 0.738 | AUC: 0.714
Fold:3
epoch:
26
Loss: 0.821 | Acc: 0.742 | BAcc: 0.741 | SEN: 0.762 | SPE: 0.719 | PRE: 0.750 | F1: 0.756 | AUC: 0.741
Fold:4
epoch:
26
Loss: 2.806 | Acc: 0.525 | BAcc: 0.548 | SEN: 0.095 | SPE: 1.000 | PRE: 1.000 | F1: 0.174 | AUC: 0.548
Fold:5
epoch:
26
Loss: 0.847 | Acc: 0.773 | BAcc: 0.762 | SEN: 0.952 | SPE: 0.571 | PRE: 0.714 | F1: 0.816 | AUC: 0.762
Resultat final:
Loss: 2.426 | Acc: 0.646 | BAcc: 0.654 | SEN: 0.515 | SPE: 0.794 | PRE: 0.838 | F1: 0.503 | AUC: 0.654
Training...:/content/drive/MyDrive/TFG_DATA/CNvsMCI/5fold/
Configuracio:
['Densenet201', 32, 0.0001, 6]
Resultat:
Fold:1
epoch:
6
Loss: 0.464 | Acc: 0.733 | BAcc: 0.7